In [ ]:
import os
import sys
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD

In [ ]:

# some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 3000
EMBEDDING_DIM = 400
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 1000
LATENT_DIM = 25

In [ ]:
import pandas as pd
dataset = pd.read_pickle('/content/drive/MyDrive/Colab_data/all_sequence.p')

In [ ]:
dataset = dataset[1:40000]

In [ ]:
dataset[2]

'Các con đã biết bản ngắn'

In [ ]:
# load in the data
import re
input_texts = []
target_texts = []
cleaned_dataset = []

for line in dataset:
  if not line:
    continue
  cleaned = re.sub(r'\W+', ' ', line).lower()

  if not words[-1]:
    continue

  words = cleaned.split()
  
  if not words[-1]:
    continue
  input_line = words[0:-1]
  target_line = words[-1]

  input_texts.append(input_line)
  target_texts.append(target_line)
  cleaned_dataset.append(cleaned)



In [ ]:
print(input_texts[123])
print(target_texts[123])
print(cleaned_dataset[123])
print(max(len(s) for s in cleaned_dataset))

['buồm', 'lướt']
ván
buồm lướt ván 
42


In [ ]:
print(target_texts)

['chuyện', 'nào', 'ngắn', 'con', 'hơn', 'người', 'mẹ', 'đây', 'điển', 'điển', 'mừng', 'thân', 'trị', 'york', 'làm', 'đây', 'trái', 'nửa', 'nửa', 'nữa', 'trước', 'sàng', 'sàng', 'hay', 'thúc', 'phục', 'thì', 'nhật', 'tay', 'vào', 'đây', 'nassau', 'khảo', 'phiếu', 'sàng', 'hộ', 'nữ', 'môn', 'xuống', 'chứ', 'sàng', 'tốt', 'montreal', 'luôn', 'xiếc', 'nào', 'rồi', 'robin', 'người', 'qua', 'gael', 'chào', 'rồi', 'ấy', 'bình', 'đẹp', 'sao', 'không', 'tay', 'sao', 'hậu', 'ta', 'rằng', 'tay', 'thua', 'đúng', 'thi', 'giới', 'thi', 'đấy', 'lil', 'cho', 'nhỉ', 'gael', 'à', 'gael', 'kyle', 'gael', 'girl', 'eo', 'vẻ', 'mình', 'vui', 'phúc', 'khác', 'tôi', 'à', 'khốn', 'này', 'nữ', 'đâu', 'tư', 'đấy', 'xa', 'nối', 'thận', 'tai', 'anh', 'ấm', 'ra', 'nhiên', 'nghề', 'nghề', 'mỹ', 'sống', 'buồm', 'hệ', 'lúc', 'người', 'nào', 'đi', 'nào', 'hệ', 'được', 'biết', 'lily', 'sách', 'tiện', 'không', 'hệ', 'đó', 'thử', 'loại', 'ván', 'thể', 'loại', 'tuyết', '11', 'cuối', 'smithsonian', 'này', 'rồi', 'rồi', 'thắ

In [ ]:
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(cleaned_dataset)
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)
word2idx = tokenizer.word_index

In [ ]:
#print(input_sequences[123])
empty_list = []
for i,x in enumerate(target_sequences):
  if len(x)==0:
    empty_list.append(i)
  
print(len(empty_list))


0


In [ ]:
for x in empty_list:
  print(target_texts[x])
  strr = target_texts[x]
  print(word2idx[strr])

nassau
3473
girl
3475
eo
3476
smithsonian
3478
aires
3481
tyler
3483
retriever
3489
cadillac
3497
hippy
3502
thiu
3504
himalaya
3509
5459
3510
a3932
3511
nobel
3513
laptop
3516
skinz
3518
brook
3523
rãnh
3525
holland
3526
bím
3527
2029
3530
center
3532
raymond
3535
canucks
3536
thính
3540
poe
3546
youtube
3550
nascar
3556
golf
3559
vãng
3560
funkapalooza
3563
kristoff
3565
caliente
3567
312
3570
strokes
3578
lõa
3585
compuserve
3587
sheila
3591
sầu
3595
roswell
3601
spam
3602
thé
3604
1983
3605
ngừ
3607
67
3610
1972
3611
chạ
3615
joyce
3619
1982
3620
everglades
3621
ngoặt
3625
five
3627
fonda
3629
cáy
3630
jasmine
3632
mổ
3636
jeannie
3642
nghệch
3644
arvydas
3645
cage
3653
nặc
3654
powers
3656
nã
3660
popeye
3661
quest
3665
geraldine
3667
xát
3669
pepper
3671
ngưu
3673
glee
3680
toa
3686
mốc
3687
hccqkl
3689
lyme
3690
lóa
3000
280
3708
goodwill
3716
dúa
3717
desalvo
3718
bingo
3008
rascal
3009
bingo
3008
xỉ
3725
supertramp
3730
me
3735
gốm
3011
riddler
3014
gumby
3015
cafein
3743
neve

In [ ]:
num_words = len(word2idx)

In [ ]:
# find max seq length


max_sequence_length_from_data = max(len(s) for s in input_sequences)
print('Max sequence length:', max_sequence_length_from_data)
# get word -> integer mapping
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))


Max sequence length: 11
Found 4984 unique tokens.


In [ ]:
max((s) for s in input_sequences)

[4982, 2090, 28, 500, 150]

In [ ]:
max_sequence_length = min(max_sequence_length_from_data, MAX_SEQUENCE_LENGTH)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
print('Shape of data tensor:', input_sequences.shape)


Shape of data tensor: (39999, 11)


In [ ]:
#from gensim.models import KeyedVectors
#word2vec = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Colab_data/NLP_embeddinglayer/wiki.vi.model.bin.gz", binary=True)

In [ ]:
num_words = len(word2idx) + 1
print(num_words)

4985


In [ ]:
print(input_sequences[0])
print(target_sequences[0])

[ 49  30   5 137 114  10 140   0   0   0   0]
[40]


In [ ]:
from tensorflow.keras.utils import to_categorical
target_sequences = to_categorical(target_sequences, num_classes=num_words)

In [ ]:

target_sequences.shape

(39999, 4985)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import Adam
import tensorflow as tf

checkpoint_filepath = '/content/drive/MyDrive/Colab_checkpoint/Word_Prediction'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)


model = Sequential()
model.add(Embedding(num_words, max_sequence_length, input_length=max_sequence_length))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(num_words, activation='softmax'))
# compiling the networkfrom keras.optimizers import Adam
adam = Adam(lr = 0.0001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
his = model.fit(input_sequences,target_sequences,epochs=500,verbose=1,callbacks=[model_checkpoint_callback])

Epoch 1/500
1250/1250 [==============================] - 12s 8ms/step - loss: 7.2797 - accuracy: 0.0279
Epoch 2/500
1250/1250 [==============================] - 9s 7ms/step - loss: 6.1928 - accuracy: 0.0287
Epoch 3/500
1250/1250 [==============================] - 9s 7ms/step - loss: 6.1346 - accuracy: 0.0297
Epoch 4/500
1250/1250 [==============================] - 9s 7ms/step - loss: 6.1096 - accuracy: 0.0309
Epoch 5/500
1250/1250 [==============================] - 9s 8ms/step - loss: 6.0986 - accuracy: 0.0299
Epoch 6/500
1250/1250 [==============================] - 9s 7ms/step - loss: 6.0677 - accuracy: 0.0309
Epoch 7/500
1250/1250 [==============================] - 10s 8ms/step - loss: 6.0840 - accuracy: 0.0290
Epoch 8/500
1250/1250 [==============================] - 9s 7ms/step - loss: 6.0790 - accuracy: 0.0289
Epoch 9/500
1250/1250 [==============================] - 9s 8ms/step - loss: 6.0573 - accuracy: 0.0314
Epoch 10/500
1250/1250 [==============================] - 10s 8ms/step 

KeyboardInterrupt: ignored

In [ ]:
adam = Adam(lr = 0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
his = model.fit(input_sequences,target_sequences,epochs=500,verbose=1,callbacks=[model_checkpoint_callback])
import pickle
pickle.dump(his.his,'/content/drive/MyDrive/Colab_history/his_0304')

Epoch 1/500
1250/1250 [==============================] - 12s 8ms/step - loss: 5.6563 - accuracy: 0.0495
Epoch 2/500
1250/1250 [==============================] - 9s 8ms/step - loss: 5.5039 - accuracy: 0.0532
Epoch 3/500
1250/1250 [==============================] - 9s 7ms/step - loss: 5.3981 - accuracy: 0.0566
Epoch 4/500
1250/1250 [==============================] - 9s 7ms/step - loss: 5.2496 - accuracy: 0.0680
Epoch 5/500
1250/1250 [==============================] - 9s 7ms/step - loss: 5.1510 - accuracy: 0.0758
Epoch 6/500
1250/1250 [==============================] - 9s 8ms/step - loss: 5.0362 - accuracy: 0.0820
Epoch 7/500
1250/1250 [==============================] - 9s 7ms/step - loss: 4.9416 - accuracy: 0.0923
Epoch 8/500
1250/1250 [==============================] - 9s 8ms/step - loss: 4.8031 - accuracy: 0.1029
Epoch 9/500
1250/1250 [==============================] - 10s 8ms/step - loss: 4.6744 - accuracy: 0.1186
Epoch 10/500
1250/1250 [==============================] - 9s 7ms/step -

In [ ]:
import pickle
pickle.dump(his.history,open('/content/drive/MyDrive/Colab_history/his_0304','wb'))

In [ ]:
his1 = model.fit(input_sequences,target_sequences,epochs=100,verbose=1,callbacks=[model_checkpoint_callback])


Epoch 1/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0700 - accuracy: 0.7130
Epoch 2/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0460 - accuracy: 0.7207
Epoch 3/100
1250/1250 [==============================] - 10s 8ms/step - loss: 1.0649 - accuracy: 0.7135
Epoch 4/100
1250/1250 [==============================] - 10s 8ms/step - loss: 1.0953 - accuracy: 0.7080
Epoch 5/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0697 - accuracy: 0.7121
Epoch 6/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0964 - accuracy: 0.7054
Epoch 7/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0835 - accuracy: 0.7093
Epoch 8/100
1250/1250 [==============================] - 10s 8ms/step - loss: 1.0830 - accuracy: 0.7098
Epoch 9/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0669 - accuracy: 0.7148
Epoch 10/100
1250/1250 [==============================] - 10s 8ms/step

In [ ]:
his1 = model.fit(input_sequences,target_sequences,epochs=100,verbose=1,callbacks=[model_checkpoint_callback])



Epoch 1/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0311 - accuracy: 0.7231
Epoch 2/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0220 - accuracy: 0.7239
Epoch 3/100
1250/1250 [==============================] - 10s 8ms/step - loss: 1.0382 - accuracy: 0.7234
Epoch 4/100
1250/1250 [==============================] - 10s 8ms/step - loss: 1.0289 - accuracy: 0.7226
Epoch 5/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0387 - accuracy: 0.7237
Epoch 6/100
1250/1250 [==============================] - 10s 8ms/step - loss: 1.0463 - accuracy: 0.7188
Epoch 7/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0333 - accuracy: 0.7217
Epoch 8/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0519 - accuracy: 0.7175
Epoch 9/100
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0029 - accuracy: 0.7286
Epoch 10/100
1250/1250 [==============================] - 10s 8ms/step

NameError: ignored

In [ ]:
pickle.dump(his1.history,open('/content/drive/MyDrive/Colab_history/his_0304_2','wb'))

In [ ]:
his2 = model.fit(input_sequences,target_sequences,epochs=300,verbose=1,callbacks=[model_checkpoint_callback])



Epoch 1/300
1250/1250 [==============================] - 9s 7ms/step - loss: 0.9943 - accuracy: 0.7331
Epoch 2/300
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0280 - accuracy: 0.7235
Epoch 3/300
1250/1250 [==============================] - 9s 7ms/step - loss: 0.9936 - accuracy: 0.7316
Epoch 4/300
1250/1250 [==============================] - 9s 8ms/step - loss: 0.9731 - accuracy: 0.7357
Epoch 5/300
1250/1250 [==============================] - 9s 7ms/step - loss: 1.0893 - accuracy: 0.7161
Epoch 6/300
1250/1250 [==============================] - 9s 7ms/step - loss: 0.9767 - accuracy: 0.7377
Epoch 7/300
1250/1250 [==============================] - 9s 7ms/step - loss: 0.9426 - accuracy: 0.7459
Epoch 8/300
1250/1250 [==============================] - 9s 7ms/step - loss: 0.9967 - accuracy: 0.7313
Epoch 9/300
1250/1250 [==============================] - 9s 7ms/step - loss: 0.9886 - accuracy: 0.7316
Epoch 10/300
1250/1250 [==============================] - 9s 7ms/step - l

In [ ]:
pickle.dump(his2.history,open('/content/drive/MyDrive/Colab_history/his_0304_3','wb'))
model.save('/content/drive/MyDrive/Colab_models/NLP/word_pre_74.h5')

In [ ]:
my_input = input().strip().lower()
my_input = tokenizer.texts_to_sequences([my_input])
print(my_input)
my_input = pad_sequences(my_input, maxlen=max_sequence_length, padding='post')
print(my_input)
i = (model.predict(my_input)[0]).argmax()
print(i)
pred_word = tokenizer.index_word[i]
print("Next word suggestion:",pred_word)


ngày mai là ngày quan trọng nên chúng ta
[[122, 325, 4, 122, 136, 225, 73, 45, 21]]
[[122 325   4 122 136 225  73  45  21   0   0]]
19
Next word suggestion: đi
